In [6]:
import requests
import json
import cx_Oracle 
import pandas as pd
import pandas.io.sql as sqlio
import json
import urllib3
import csv
urllib3.disable_warnings()

In [7]:
def sql_connection():

 uid = "*******"    # usuário
 pwd = "******"   # senha
 db = "******"  # string de conexão do Oracle, configurado no
                # cliente Oracle, arquivo tnsnames.ora
 connection = cx_Oracle.connect(uid+"/"+pwd+"@"+db) #cria a conexão

 return connection


In [9]:
def query_to_df(query):
 connection = sql_connection()
 data_set = sqlio.read_sql_query(query, connection)
 return data_set


query = 'SELECT * FROM Consulta_CNPJ '
df_cnpj = query_to_df(query)

In [ ]:
def get_cnpj_data(cnpj):
    # Dado um CNPJ, faz uma requisição para a API Minha Receita. Caso a requisição seja bem sucedida, retorna o conteúdo da requisição em formato json
    minha_receita_api_url = 'https://minhareceita.org/'
    r = requests.post(minha_receita_api_url, data=cnpj, verify=False)
    if r.status_code == 200:
        return json.loads(r.content)

In [ ]:
def clean_text(full_text):
    # Limpa um texto, retirando quebras de linha e ponto-e-vírgulas
    clean_text = full_text.replace("\n", " ")
    clean_text = full_text.replace(";", " ")
    return clean_text


def cnpj_data_to_df(df_cnpj):
    # Recebe um dataframe contendo os CNPJ's e, a partir das requisições à API Minha Receita, estrutura um dataframe contendo os dados do CNPJ
    data_set = pd.DataFrame(columns=[
        'cnpj',
        'razao_social',
        'nome_fantasia',
        'atividade_principal_codigo',
        'atividade_principal_descricao',
        'situacao_cadastral',
        'capital_social',
        'porte',
        'codigo_natureza_juridica',
        'data_abertura',
        'cep',
        'municipio',
        'uf',
        'nome_socio',
        'cnpj_cpf_do_socio',
        'qualificacao_socio',
        'data_entrada_sociedade'
    ])
    for cnpj in df_cnpj['NUM_CGC']:
        cnpj_data = get_cnpj_data({'cnpj': cnpj})
        df_cnpj_data = json.loads(json.dumps(cnpj_data['qsa']))
        if df_cnpj_data != None:
            for i in df_cnpj_data:
                if i != None:
                        new_row = {
                            'cnpj': cnpj,       
                            'razao_social': cnpj_data['razao_social'],
                            'nome_fantasia': cnpj_data['nome_fantasia'],
                            'atividade_principal_codigo': cnpj_data['cnae_fiscal'],
                            'atividade_principal_descricao': clean_text(cnpj_data['cnae_fiscal_descricao']),
                            'situacao_cadastral': cnpj_data['descricao_situacao_cadastral'],
                            'capital_social': float(cnpj_data['capital_social']),
                            'porte': cnpj_data['descricao_porte'],
                            'codigo_natureza_juridica': int(cnpj_data['codigo_natureza_juridica']),
                            'data_abertura': cnpj_data['data_inicio_atividade'],
                            'cep': cnpj_data['cep'],
                            'municipio': cnpj_data['municipio'],
                            'uf': cnpj_data['uf'],
                            'nome_socio': i['nome_socio'],
                            'cnpj_cpf_do_socio': i['cnpj_cpf_do_socio'],
                            'qualificacao_socio': i['qualificacao_socio'],
                            'data_entrada_sociedade': i['data_entrada_sociedade']
                                                                        }
                else:
                    new_row = {'cnpj': cnpj}
                data_set = data_set.append(new_row, ignore_index=True)
    return data_set

In [ ]:
def df_to_csv(csv_path, data_set):
    # Exporta um dataframe para um arquivo csv
    data_set.to_csv(csv_path, header=False, index=False, sep=';',
                    encoding='utf-8', quoting=csv.QUOTE_MINIMAL)

In [ ]:
def df_to_csv(csv_path, data_set):
    # Exporta um dataframe para um arquivo csv
    data_set.to_csv(csv_path, encoding='latin-1', index=False, sep=';',
                    quoting=csv.QUOTE_MINIMAL)

In [ ]:
urllib3.disable_warnings()
csv_path = 'temp_cnpj_data_teste.csv'
df_cnpj_data = pd.DataFrame()
for i in range(0, len(df_cnpj), 1000):
    new_batch = cnpj_data_to_df(df_cnpj[i:i+1000])
    df_to_csv(csv_path, new_batch)
   # csv_to_sql_table(csv_path)
    df_cnpj_data = df_cnpj_data.append(new_batch, ignore_index=True)
    print(i)